In [1]:
import generate

[Taichi] version 1.1.3, llvm 10.0.0, commit 1262a70a, osx, python 3.9.12
[I 11/08/22 21:17:44.324 10956830] [shell.py:_shell_pop_print@33] Graphical python shell detected, using wrapped sys.stdout
[Taichi] Starting on arch=metal


In [81]:
class Camera:
    def __init__(self, fov=60, aspect_ratio=1.0):
        # Camera parameters
        self.lookfrom = np.array((0.4, 0.5, 2.0))
        self.lookat = np.array([0.0, 0.0, 0.0])
        self.vup = np.array([0.0, 1.0, 0.0])
        self.fov = fov
        self.aspect_ratio = aspect_ratio
        
        theta = self.fov * (PI / 180.0)
        half_height = ti.tan(theta / 2.0)
        half_width = self.aspect_ratio * half_height
        
        self.cam_origin = self.lookfrom
        
        w = self.lookfrom - self.lookat
        w = w / np.linalg.norm(w)

        u = np.cross(self.vup, w)
        u = u / np.linalg.norm(u)
        
        v = np.cross(w, u)
        
        self.cam_lower_left_corner = self.cam_origin - half_width * u - half_height * v - w
        self.cam_horizontal = 2 * half_width * u
        self.cam_vertical = 2 * half_height * v
    
    def test(self):
        print("test")
    
    def get_direction(self, u, v):
        
        u = (u) / 800
        v = (v) / 800

        direction = self.cam_lower_left_corner + u * self.cam_horizontal + v * self.cam_vertical - self.cam_origin
        #direction = direction / np.linalg.norm(direction)
        
        print("self.cam_lower_left_corner[None]", self.cam_lower_left_corner)
        print("u", u)
        print("self.cam_horizontal[None]", u * self.cam_horizontal)
        print("self.cam_vertical[None]", v * self.cam_vertical)
        print("self.cam_origin[None]", self.cam_origin)
        
        return direction
    
    def get_impossible_intersection(self,index,u,v):
        direction = self.get_direction(u,v)
        return self.lookfrom + index*direction

In [85]:
a = Camera()

a.get_impossible_intersection(3,400, 500)

self.cam_lower_left_corner[None] [-0.32965573 -0.29884196  1.29564164]
u 0.5
self.cam_horizontal[None] [ 0.56613852  0.         -0.1132277 ]
self.cam_vertical[None] [-0.03369872  0.7009334  -0.16849361]
self.cam_origin[None] [0.4 0.5 2. ]


array([-0.1916478 ,  0.20627433, -0.95823902])

In [88]:
a.get_impossible_intersection(4,400, 500)

self.cam_lower_left_corner[None] [-0.32965573 -0.29884196  1.29564164]
u 0.5
self.cam_horizontal[None] [ 0.56613852  0.         -0.1132277 ]
self.cam_vertical[None] [-0.03369872  0.7009334  -0.16849361]
self.cam_origin[None] [0.4 0.5 2. ]


array([-0.38886374,  0.10836577, -1.94431869])

In [84]:
a.get_direction(400, 500)

self.cam_lower_left_corner[None] [-0.32965573 -0.29884196  1.29564164]
u 0.5
self.cam_horizontal[None] [ 0.56613852  0.         -0.1132277 ]
self.cam_vertical[None] [-0.03369872  0.7009334  -0.16849361]
self.cam_origin[None] [0.4 0.5 2. ]


array([-0.19721593, -0.09790856, -0.98607967])